# ELBO Gradient Estimators

这一部分是来告诉我们`pyro`是如何估计ELBO的梯度的。

$$
∇θ,ϕELBO=∇θ,ϕEqϕ(z)[logpθ(x,z)−logqϕ(z)]
$$

## 简单情况：Reparameterizable Random Variables

其实我们可以从上面的公式中看到，最麻烦的地方是求期望这个部分会存在需要计算梯度的参数phi。

如果我们能够进行下面的改变：

$$\mathbb{E}_{q_{\phi}(Z)}[f_{\phi}(Z)]=\mathbb{E}_{q(\epsilon)}[f_{\phi}(g_{\phi}(\epsilon))]$$

即将求期望里面的可训练参数去掉，$q(\epsilon)$是一个确定的分布。则可以进行如下的计算：

$$\bigtriangledown_{\phi}{\mathbb{E}_{q(\epsilon)}[f_{\phi}(g_{\phi}(\epsilon))]}=\mathbb{E}_{q(\epsilon)}[\bigtriangledown_{\phi}{f_{\phi}(g_{\phi}(\epsilon))}]$$

使用简单的Monte Carlo估计即可。在这种情况下，model只需要能够计算pdf，guide能够进行采样即可。

## 复杂情况：Non-reparameterizable Random Variables

这种情况也是大部分存在的情况，比如所有的离散分布。

我们进行下面的变化：

$$
\bigtriangledown_{\phi}{\mathbb{E}_{q_{\phi}(Z)}[f_{\phi}(Z)]}=\bigtriangledown_{\phi}{\int{dZq_{\phi}(Z)f_{\phi}(Z)}}
$$

然后交换求导和积分（暂且先不去考虑是否可交换），根据求导的规则，有下面的结果：

$$
\int{dZ\{(\bigtriangledown_{\phi}{q_{\phi}(Z)})f_{\phi}(Z)+q_{\phi}(Z)(\bigtriangledown_{\phi}{f_{\phi}(Z)})\}}
$$

使用下面的等式：

$$\bigtriangledown_{\phi}{q_{\phi}(Z)}=q_{\phi}(Z)\bigtriangledown_{\phi}{\log{q_{\phi}(Z)}}$$

将进一步得到：

$$\bigtriangledown_{\phi}{\mathbb{E}_{q_{\phi}(Z)}[f_{\phi}(Z)]}=\mathbb{E}_{q_{\phi}(Z)}[(\bigtriangledown_{\phi}{\log{q_{\phi}(Z)}})f_{\phi}(Z)+\bigtriangledown_{\phi}{f_{\phi}(Z)}]$$

以上的梯度估计也被称为**REINFORCE估计**，可以借助Monte Carlo来进行计算。这里进一步提示：model需要能够计算pdf即可，而guide除了能够计算pdf还需要能够进行采样。

其中，被求梯度的项：

$${\log{q_{\phi}(Z)}}\overline{f_{\phi}(Z)}+f_{\phi}(Z)$$

称为surrogate objective。

## Variance

但实际上我们不能直接使用上面估计，因为实践中显示这个估计的方差实在是太大了，从而根本无法进行有效的工作。所以我们必须积极寻求降低此估计方差的办法，注意有以下两种测量：

第一种，**通过依赖结构**。

假设我们有下面的cost function的形式：

$$f_{\phi}(Z)=\log{p_{\theta}(X|Pa_p(X))}+\sum_i{\log{p_{\theta}(Z_i|Pa_p(Z_i))}}-\sum_i{\log{q_{\phi}(Z_i|Pa_q(Z_i))}}$$

再经过一系列的推导，其会将一些项给删去，从而降低了方差。

是实际使用中，只需要将`TraceGraph_ELBO`替换`Trace_ELBO`即可。但注意，这个会进行额外的计算，所以只在有无法进行重参数化技巧的问题上使用，正常使用`Trace_ELBO`即可。

第二种，**通过数据依赖基准（Data-Dependent Baselines）**

基本思路和上面一致，但并不是去减少一些期望为0的项，而是添加一些特殊项从而减小方差。添加的那一项（一般为常数）就是Baselines。

一般的形式是：

$${\log{q_{\phi}(Z)}}(\overline{f_{\phi}(Z)}-b)$$

在pyro中，我们需要为每个以Z设置baseline，所以需要在`pyro.sample`上设置，其拥有参数`infer`，其接受dict，其中一个key可以是`baseline`，然后其value是dict，包括一些真正关于baseline的设置。

```python
z = pyro.sample("z", dist.Bernoulli(...),
                infer=dict(baseline={'use_decaying_avg_baseline': True,
                                     'baseline_beta': 0.95}))
```

## Example

这还是那个coins反转的例子，只是这里不再将Beta分布视为可以重参数化的分布，而是使用baselines技巧来实现。

In [2]:
import os
import sys
import torch
import torch.distributions.constraints as constraints
import pyro
import pyro.distributions as dist
from pyro.distributions.testing.fakes import NonreparameterizedBeta
import pyro.optim as optim
from pyro.infer import SVI, TraceGraph_ELBO

def param_abs_error(name, target):
    return torch.sum(torch.abs(target - pyro.param(name))).item()

class BernoulliBetaExample:
    def __init__(self, max_steps):
        # the maximum number of inference steps we do
        self.max_steps = max_steps
        # the two hyperparameters for the beta prior
        self.alpha0 = 10.0
        self.beta0 = 10.0
        # the dataset consists of six 1s and four 0s
        self.data = torch.zeros(10)
        self.data[0:6] = torch.ones(6)
        self.n_data = self.data.size(0)
        # compute the alpha parameter of the exact beta posterior
        self.alpha_n = self.data.sum() + self.alpha0
        # compute the beta parameter of the exact beta posterior
        self.beta_n = - self.data.sum() + torch.tensor(self.beta0 + self.n_data)
        # initial values of the two variational parameters
        self.alpha_q_0 = 15.0
        self.beta_q_0 = 15.0

    def model(self, use_decaying_avg_baseline):
        # sample `latent_fairness` from the beta prior
        f = pyro.sample("latent_fairness", dist.Beta(self.alpha0, self.beta0))
        # use plate to indicate that the observations are
        # conditionally independent given f and get vectorization
        with pyro.plate("data_plate"):
            # observe all ten datapoints using the bernoulli likelihood
            pyro.sample("obs", dist.Bernoulli(f), obs=self.data)

    def guide(self, use_decaying_avg_baseline):
        # register the two variational parameters with pyro
        alpha_q = pyro.param("alpha_q", torch.tensor(self.alpha_q_0),
                             constraint=constraints.positive)
        beta_q = pyro.param("beta_q", torch.tensor(self.beta_q_0),
                            constraint=constraints.positive)
        # sample f from the beta variational distribution
        baseline_dict = {'use_decaying_avg_baseline': use_decaying_avg_baseline,
                         'baseline_beta': 0.90}
        # note that the baseline_dict specifies whether we're using
        # decaying average baselines or not
        pyro.sample("latent_fairness", NonreparameterizedBeta(alpha_q, beta_q),
                    infer=dict(baseline=baseline_dict))

    def do_inference(self, use_decaying_avg_baseline, tolerance=0.80):
        # clear the param store in case we're in a REPL
        pyro.clear_param_store()
        # setup the optimizer and the inference algorithm
        optimizer = optim.Adam({"lr": .0005, "betas": (0.93, 0.999)})
        svi = SVI(self.model, self.guide, optimizer, loss=TraceGraph_ELBO())
        print("Doing inference with use_decaying_avg_baseline=%s" % use_decaying_avg_baseline)

        # do up to this many steps of inference
        for k in range(self.max_steps):
            svi.step(use_decaying_avg_baseline)
            if k % 100 == 0:
                print('.', end='')
                sys.stdout.flush()

            # compute the distance to the parameters of the true posterior
            alpha_error = param_abs_error("alpha_q", self.alpha_n)
            beta_error = param_abs_error("beta_q", self.beta_n)

            # stop inference early if we're close to the true posterior
            if alpha_error < tolerance and beta_error < tolerance:
                break

        print("\nDid %d steps of inference." % k)
        print(("Final absolute errors for the two variational parameters " +
               "were %.4f & %.4f") % (alpha_error, beta_error))

# enable validation (e.g. validate parameters of distributions)
assert pyro.__version__.startswith('1.3.0')
pyro.enable_validation(True)

# this is for running the notebook in our testing framework
smoke_test = ('CI' in os.environ)
max_steps = 2 if smoke_test else 10000
bbe = BernoulliBetaExample(max_steps=max_steps)
bbe.do_inference(use_decaying_avg_baseline=True)
bbe.do_inference(use_decaying_avg_baseline=False)

Doing inference with use_decaying_avg_baseline=True
..
Did 167 steps of inference.
Final absolute errors for the two variational parameters were 0.7943 & 0.7986
Doing inference with use_decaying_avg_baseline=False
.........
Did 878 steps of inference.
Final absolute errors for the two variational parameters were 0.7978 & 0.7138


上述结果显示，baselines的加入使得我们训练的steps减少了许多，提高了效率。